In [2]:
import yfinance as yf
import investpy

import matplotlib.pyplot as plt

from datetime import datetime
import pandas as pd

import numpy as np

#https://github.com/ranaroussi/yfinance

#pip install git+https://github.com/alvarobartt/investpy@403-patch
#https://github.com/alvarobartt/investpy/issues/600

In [3]:
# 1 run py script
is_py=False
#is_py=True

# 2 manual update new symbol
is_new=False
#is_new=True

# 3. select only one thai or FIF
new_x=''
new_x='investing' 
#new_x='yahoo'

#1 = yahoo  , 2=invesing ,3=all
option=2

#start_param="2016-01-01"
start_param="2000-09-01"
end_param=end= datetime.now().strftime("%Y-%m-%d")


In [4]:
# yfinanace for world fund
list_cols=['Ticker','Date','Open','High','Low','Close','Volume','Name']
list_cols_number=['Open','High','Low','Close','Volume']

# investing.com for thai fund
Cols=['Ticker','Date','Open','High','Low','Close','Volumne','Name']

data_path=r'D:\AB_DB\Script_ImportData\\Fund_File'
import_file=f"{data_path}\\List_Fund_Import.xlsx"

world_file=f'{data_path}\\AB_Fund_World.csv'
new_world_file=f'{data_path}\\New\\AB_Fund_World_NewEntry.csv'
thai_file=f'{data_path}\\AB_Fund_THAI.csv'
new_thai_file=f'{data_path}\\New\\AB_Fund_THAI_New.csv'

In [5]:
if is_py:
    print("Selec Option : 1=Yahoo,2=Investing and 3=Both")
    option_param = input("Enter start : ")
    option=int(option_param)

    
    print("Enter start date as yyyy.mm.dd(2019-01-01)")
    start_param = input("Enter start : ")

    print("Enter end date as yyyy.mm.dd(2022-01-01) if today ,press enter")
    end_param = input("Enter end : ") or ''

    start=start_param
    if end_param == '':
     end_param=datetime.now().strftime("%Y-%m-%d")
    
if new_x=='investing' and is_new==True: 
 new_sheet="NewInvesting" 
elif new_x=='yahoo' and is_new==True:
 new_sheet="NewYahoo"     

print(start_param)
print(end_param)





    

2000-09-01
2022-09-23


# yfinance

In [6]:
# get price from yahoo
# https://aroussi.com/post/python-yahoo-finance
# https://github.com/ranaroussi/yfinance
if option==1 or option==3:

    print("################################Start Download from Yahoo========================================")
    if is_new==True and new_x=='yahoo':
        print("New Yahoo")
        df_ticker=pd.read_excel(import_file,sheet_name=new_sheet)
    else:
        df_ticker=pd.read_excel(import_file,sheet_name='Yahoo')
        print("World Investing")


    df_ticker =df_ticker.replace(np.nan, '', regex=True)
    print(df_ticker)

    list_not_found=[]

    df=pd.DataFrame(columns=list_cols)

    for index,item in df_ticker.iterrows():

       try:

        ticker=item['Ticker']
        name=item['Name']
        print(f'{ticker} {name}')

        dfx = yf.download(ticker, start=start_param, end=end_param)
        if (dfx.empty==False) or (dfx is not None):
            dfx['Ticker'] = ticker
            dfx['Name']=name
            dfx=dfx.reset_index()
            dfx=dfx[list_cols]
            df=pd.concat([df,dfx])

            print(dfx.tail())
        else:
            print(f"{ticker} No data found for this date range, symbol may be delisted")
            list_not_found.append(ticker) 


       except Exception as ex:
          print(str(ex))
          list_not_found.append(ticker)  


    # df=df.rename(columns={'Adj Close':'Close'})

    df[list_cols_number]=df[list_cols_number].round(2)
    print("Save file as csv.")
    if is_new==True and new_x=='yahoo':
        df.to_csv(new_world_file,index=False)
        print(new_world_file)
    else:
        df.to_csv(world_file,index=False)
        print(world_file)


    print("################################End Download from Yahoo========================================")

    print("World fund : Not found symbol")
    list_not_found


# Investpy

In [7]:
if option==1:
 print("Download only yahoo")   
 quit()

In [8]:
print("################################Start Download from Investing.com========================================")

# for investpy
start_date=datetime.strptime(start_param, '%Y-%m-%d').strftime('%d/%m/%Y')
to_date=datetime.strptime(end_param, '%Y-%m-%d').strftime('%d/%m/%Y')
print(start_date)
print(to_date)

if is_new==True and new_x=='investing':
    thai_ticker=pd.read_excel(import_file,sheet_name=new_sheet)
    print("New ThaiFund")
else:
    thai_ticker=pd.read_excel(import_file,sheet_name='Investing')
    print("ThaiFund")
       
thai_ticker =thai_ticker.replace(np.nan, '', regex=True)

thai_ticker['Name']=thai_ticker['Name'].apply(lambda x: x.strip())
thai_ticker['Ticker']=thai_ticker['Ticker'].apply(lambda x: x.strip())
print(thai_ticker)

################################Start Download from Investing.com========================================
01/09/2000
23/09/2022
ThaiFund
                                                 Name         Ticker  \
0                               Buakaew Open-end Fund            BKA   
1                    Bualuang Global Health Care Fund          BCARE   
2                                  Bualuang Gold Fund          BGOLD   
3   Fidelity Funds - Global Technology Fund A-acc-usd     0P00012JRK   
4   Allianz Global Investors Fund - Allianz Global...     0P0001A3R3   
5   Baillie Gifford Positive Change Fund B Accumul...     0P0001BDIT   
6   Goldman Sachs Global Core Equity Portfolio I A...     0P000081VE   
7                               Krungsri Dynamic Fund      KFDYNAMIC   
8             Krungsri Thai Small-mid Cap Equity Fund       KFTHAISM   
9                                 Krungsri Set100 Rmf      KFS100RMF   
10               Phatra Equity Retirement Mutual Fund     KKP ACT EQ   

In [9]:
thai_not_found=[]

df_thai=pd.DataFrame(columns=Cols)
print(df_thai)
for index, row in thai_ticker.iterrows():
 try:
    name=row['Name']
    ticker=row['Ticker']
    country=row["Country"]
    print(f'{ticker} {name}')
    
    
    df = investpy.get_fund_historical_data(fund=name,
                                            country=country,
                                            from_date=start_date,
                                            to_date=to_date)
    if (df.empty ==False)  or (df is not None):
        df['Ticker']=ticker
        df['Name']=name
        df['Volumne']=100
        df=df.reset_index()
        df=df[Cols]
      

        df_thai=pd.concat([df_thai,df])
        
        print(df.tail())
        
    else:
        print(f"{ticker} No data found for this date range, symbol may be delisted")
        list_not_found.append(ticker) 
    
 except Exception as ex:
    thai_not_found.append(ticker) 
    print(str(ex))

 
df_thai=df_thai.round(2)
df_thai.head()


print("Save file as csv.")
if is_new==True and new_x=='investing':
    df_thai.to_csv(new_thai_file,index=False)
    print(new_thai_file)
else:
    df_thai.to_csv(thai_file,index=False)
    print(thai_file)
    
print("################################End Download from Investing.com========================================")

print("Thai fund : Not found symbol")
thai_not_found


Empty DataFrame
Columns: [Ticker, Date, Open, High, Low, Close, Volumne, Name]
Index: []
BKA Buakaew Open-end Fund
ERR#0015: error 403, try again later.
BCARE Bualuang Global Health Care Fund
ERR#0015: error 403, try again later.
BGOLD Bualuang Gold Fund
ERR#0015: error 403, try again later.
0P00012JRK Fidelity Funds - Global Technology Fund A-acc-usd
ERR#0015: error 403, try again later.
0P0001A3R3 Allianz Global Investors Fund - Allianz Global Artificial Intelligence At Usd
ERR#0015: error 403, try again later.
0P0001BDIT Baillie Gifford Positive Change Fund B Accumulation
ERR#0015: error 403, try again later.
0P000081VE Goldman Sachs Global Core Equity Portfolio I Acc Usd Close
ERR#0015: error 403, try again later.
KFDYNAMIC Krungsri Dynamic Fund
ERR#0015: error 403, try again later.
KFTHAISM Krungsri Thai Small-mid Cap Equity Fund
ERR#0015: error 403, try again later.
KFS100RMF Krungsri Set100 Rmf
ERR#0015: error 403, try again later.
KKP ACT EQ Phatra Equity Retirement Mutual Fund

['BKA',
 'BCARE',
 'BGOLD',
 '0P00012JRK',
 '0P0001A3R3',
 '0P0001BDIT',
 '0P000081VE',
 'KFDYNAMIC',
 'KFTHAISM',
 'KFS100RMF',
 'KKP ACT EQ',
 'KKP SM CAP',
 'PHATRASG-AA',
 'KKP SMART MV',
 'K-GHEALTH(UH)',
 'K-EQUITY',
 'K-STAR-A(R)',
 'K-MIDSMALL',
 'KT-AASIA-A',
 'K-AEC',
 'K-INDX',
 'K-CHX',
 'KTEF',
 'T-SMCAP',
 'TISCOAP',
 'TSF',
 'TISCOMS',
 'TISCOEGF',
 'TISCOAGF',
 'TISCOLAF',
 'TUSREIT',
 'TMBGQG',
 'TMBCORMF',
 'TMBCHEQ',
 'TMBWDEQ',
 'TMBEMEQ',
 'TSTARP',
 'TMBAGLRMF',
 'TMB50',
 'K-SELECT']